In [2]:
!pip install chromadb llama-index llama-index-vector-stores-chroma fastembed transformers huggingface_hub torch gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 72.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s et

In [3]:
!pip install llama-index-embeddings-fastembed

In [4]:
!pip install chromadb \
    llama-index \
    llama-index-vector-stores-chroma \
    llama-index-embeddings-fastembed \
    llama-index-llms-huggingface \
    fastembed \
    transformers \
    huggingface_hub \
    torch \
    gradio

In [5]:
import logging
import sys
import torch
import chromadb
import requests
from transformers import AutoTokenizer

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.embeddings.fastembed import FastEmbedEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import PromptTemplate, StorageContext
from llama_index.vector_stores.chroma import ChromaVectorStore

In [6]:
# Logging setup
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [11]:
from huggingface_hub import notebook_login
notebook_login()

In [9]:

# Load documents
documents = SimpleDirectoryReader("/content/Data").load_data()


In [ ]:

# Initialize embedding model
embed_model = FastEmbedEmbedding(model_name="BAAI/bge-small-en-v1.5")
Settings.embed_model = embed_model
Settings.chunk_size = 512

# System prompt
system_prompt = "You are a Q&A assistant. Your goal is to answer questions as accurately as possible based on the instructions and context provided."
query_wrapper_prompt = PromptTemplate("<|USER|>{query_str}<|ASSISTANT|>")

# Load tokenizer for Meta-Llama-3-8B-Instruct
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")
stopping_ids = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

# Configure LLM
llm = HuggingFaceLLM(
    context_window=8192,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.7, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="meta-llama/Meta-Llama-3-8B-Instruct",
    model_name="meta-llama/Meta-Llama-3-8B-Instruct",
    device_map="auto",
    stopping_ids=stopping_ids,
    tokenizer_kwargs={"max_length": 4096},
    model_kwargs={"torch_dtype": torch.float16}
)
Settings.llm = llm

# Set up ChromaDB
chroma_client = chromadb.PersistentClient(path="./chroma_db")
vector_store = ChromaVectorStore(chroma_collection=chroma_client.get_or_create_collection("my_collection"))
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Build vector index
index = VectorStoreIndex.from_documents(documents, storage_context=storage_context)

# Create query engine
query_engine = index.as_query_engine()

# Google Search API Setup
GOOGLE_API_KEY = "api_key"
SEARCH_ENGINE_ID = "engine_id"

def google_cse_search(query, num_results=5):
    """Fetches search results from Google Custom Search API"""
    url = "https://www.googleapis.com/customsearch/v1"
    params = {
        "key": GOOGLE_API_KEY,
        "cx": SEARCH_ENGINE_ID,
        "q": query,
        "num": num_results
    }

    response = requests.get(url, params=params)
    data = response.json()
    results = [item["link"] for item in data.get("items", [])] if "items" in data else []

    return results

# Query Router Function
def should_use_google_search(query):
    """
    A simple heuristic-based function to decide whether to use Google Search or ChromaDB.
    You can improve this by training a classifier model or fine-tuning an LLM.
    """
    keywords_for_search = ["latest", "current", "trending", "news", "update", "recent", "today"]

    # If the query contains keywords suggesting the need for fresh information, use Google Search
    if any(keyword in query.lower() for keyword in keywords_for_search):
        return True
    return False

# Unified Query Function
def predict(input, history):
    """
    Routes the query to either ChromaDB or Google Search based on the type of question.
    """
    if should_use_google_search(input):
        search_results = google_cse_search(input)
        response = f"Based on web search, here are some useful links:\n" + "\n".join(search_results)
    else:
        response = query_engine.query(input)

    return str(response)

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

Some parameters are on the meta device because they were offloaded to the cpu.


In [13]:
# Deploy chatbot with Gradio
import gradio as gr
gr.ChatInterface(predict).launch(share=True)

/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:334: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4f5e06b8dbe7efcef9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
